In [24]:
!pip install ripser transformers accelerate torch numpy pandas tqdm networkx hf_transfer datasets seaborn pyarrow fastparquet datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached fastparquet-2024.11.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached cramjam-2.11.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached fastparquet-2024.11.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.8 MB)
Using cached cramjam-2.11.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [fastparquet]


In [2]:
from huggingface_hub import login
login()

In [25]:
# imports
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import pandas as pd
import ripser
from ripser import ripser
from tqdm import tqdm
import math
import numpy as geek
import networkx as nx
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [26]:
# tda functions
def find_highest_finite_value_comprehension(data):
    finite_values = [x for x in data if not math.isinf(x)]
    return max(finite_values) if finite_values else -math.inf

def get_second_value_ignoring_inf(data):
    non_inf_values = [x for x in data if not math.isinf(x)]
    if len(non_inf_values) < 2:
        return None
    return non_inf_values[1]

def get_llama_attention(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs, output_attentions=True)
    
    attention_matrices = torch.stack(outputs.attentions).cpu()
    
    attention_matrix = attention_matrices.mean(dim=0).mean(dim=1).squeeze().numpy()
    
    np.fill_diagonal(attention_matrix, 0)
    
    attention_matrix = (attention_matrix + attention_matrix.T) / 2
    
    return attention_matrix

def build_graph(attention_matrix, threshold=None, percentile=50):
    graph = nx.Graph()
    num_nodes = attention_matrix.shape[0]
    
    graph.add_nodes_from(range(num_nodes))
    
    upper_triangle_indices = np.triu_indices(num_nodes, k=1)
    upper_triangle_values = attention_matrix[upper_triangle_indices]
    
    non_zero_values = upper_triangle_values[upper_triangle_values > 0]
    non_zero_values = non_zero_values[~np.isnan(non_zero_values)]
    
    if threshold is None:
        if len(non_zero_values) > 0:
            try:
                threshold = np.percentile(non_zero_values, percentile)
                if np.isnan(threshold) or threshold == 0:
                    threshold = np.max(non_zero_values) * 0.1
                else:
                    print(f"Using percentile-based threshold: {threshold:.6f} ({percentile}th percentile)")
                print(f"Non-zero values: {len(non_zero_values)}/{len(upper_triangle_values)} ({100*len(non_zero_values)/len(upper_triangle_values):.1f}%)")
                print(f"Value range: [{np.min(non_zero_values):.6f}, {np.max(non_zero_values):.6f}]")
            except:
                threshold = 0.001
                print(f"Error calculating percentile, using default threshold: {threshold}")
        else:
            threshold = 0.0
            print(f"WARNING: No non-zero attention values found!")
    else:
        print(f"Using fixed threshold: {threshold}")
    
    edge_count = 0
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if attention_matrix[i, j] > threshold:
                graph.add_edge(i, j, weight=attention_matrix[i, j])
                edge_count += 1
    
    print(f"Graph created: {num_nodes} nodes, {edge_count} edges")
    
    if edge_count == 0:
        print(f"WARNING: No edges created! Threshold was {threshold:.6f}, max value was {upper_triangle_values.max():.6f}")
    
    return graph

def process_texts(texts, model_name="meta-llama/Llama-3.1-8B"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        low_cpu_mem_usage=True,
        attn_implementation="eager"
    )
    
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    data = []
    for text in tqdm(texts):
        try:
            attention_matrix = get_llama_attention(text, model, tokenizer)
            graph = build_graph(attention_matrix, threshold=None, percentile=50)
            tda_features = compute_tda_features(graph)
            data.append(tda_features)
        except Exception as e:
            print(f"Error processing text: {e}")
            data.append([0] * 12)

    columns = ["Num_0dim", "Max_0dim", "Max_0dim_Minus_Second", "Mean_0dim", "betti_curve_0", "persistence_entropy_0",
               "Num_1dim", "Max_1dim", "Max_1dim_Minus_Second", "Mean_1dim", "betti_curve_1", "persistence_entropy_1"]
    return pd.DataFrame(data, columns=columns)

def compute_tda_features(graph):
    adjacency_matrix = nx.to_numpy_array(graph)
    
    # Check if graph is empty or has no edges
    if adjacency_matrix.shape[0] == 0 or np.sum(adjacency_matrix) == 0:
        print("Empty graph, returning zero features")
        return [0] * 12
    
    # Use distance matrix for ripser
    distance_matrix = 1 - adjacency_matrix  # Convert similarity to distance
    np.fill_diagonal(distance_matrix, 0)  # Ensure diagonal is 0
    
    diagrams = ripser(distance_matrix, maxdim=1, distance_matrix=True)['dgms']
    
    h0 = diagrams[0]
    h1 = diagrams[1] if len(diagrams) > 1 else np.array([])

    num_h0 = geek.count_nonzero(np.round(h0))
    highest_h0 = find_highest_finite_value_comprehension(h0[:, 1] - h0[:, 0]) if num_h0 > 0 else 0
    Second_highest_h0 = get_second_value_ignoring_inf(h0[:, 1] - h0[:, 0]) if num_h0 > 1 else 0
    highest_minus_second_h0 = highest_h0 - Second_highest_h0 if num_h0 > 1 else 0

    h0[np.isinf(h0)] = 0
    mean_h0 = np.mean(h0) if num_h0 > 0 else 0

    num_h1 = geek.count_nonzero(np.round(h1))
    highest_h1 = find_highest_finite_value_comprehension(h1[:, 1] - h1[:, 0]) if num_h1 > 0 else 0
    second_highest_h1 = get_second_value_ignoring_inf(h1[:, 1] - h1[:, 0]) if num_h1 > 1 else 0
    highest_minus_second_h1 = highest_h1 - second_highest_h1 if num_h1 > 1 else 0

    h1[np.isinf(h1)] = 0
    mean_h1 = np.mean(h1) if num_h1 > 0 else 0

    h0_persistences = np.sort(h0[:, 1] - h0[:, 0]) if num_h0 > 1 else np.array([0])
    h1_persistences = np.sort(h1[:, 1] - h1[:, 0]) if num_h1 > 1 else np.array([0])

    sum_persistence_0 = np.sum(h0_persistences) if len(h0_persistences) > 0 else 0
    sum_persistence_1 = np.sum(h1_persistences) if len(h1_persistences) > 0 else 0
    persistence_entropy_0 = -np.sum(h0_persistences * np.log(h0_persistences + 1e-10)) if len(h0_persistences) > 0 else 0
    persistence_entropy_1 = -np.sum(h1_persistences * np.log(h1_persistences + 1e-10)) if len(h1_persistences) > 0 else 0
    betti_curve_0 = len(h0_persistences)
    betti_curve_1 = len(h1_persistences)

    return [num_h0, highest_h0, highest_minus_second_h0, mean_h0, betti_curve_0, persistence_entropy_0,
            num_h1, highest_h1, highest_minus_second_h1, mean_h1, betti_curve_1, persistence_entropy_1]

In [23]:
texts = ["This is a test sentence.", "Another example of text processing."]
df = process_texts(texts)
print(df)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00, 19.21it/s]

Attention shape: (7, 7), min: 0.000000, max: 0.471680, mean: 0.115845
Using percentile-based threshold: 0.015778 (50th percentile)
Non-zero values: 21/21 (100.0%)
Value range: [0.006222, 0.471680]
Graph created: 7 nodes, 10 edges
Attention shape: (7, 7), min: 0.000000, max: 0.476318, mean: 0.115540
Using percentile-based threshold: 0.018997 (50th percentile)
Non-zero values: 21/21 (100.0%)
Value range: [0.008690, 0.476318]
Graph created: 7 nodes, 10 edges
   Num_0dim  Max_0dim  Max_0dim_Minus_Second  Mean_0dim  betti_curve_0  \
0         7  0.611816               0.067627   0.241752              7   
1         7  0.611816               0.065674   0.242536              7   

   persistence_entropy_0  Num_1dim  Max_1dim  Max_1dim_Minus_Second  \
0               1.933568         0         0                      0   
1               1.928764         0         0                      0   

   Mean_1dim  betti_curve_1  persistence_entropy_1  
0          0              1                   -0.0

In [27]:
import pyarrow
import pyarrow.parquet as pq
from datasets import load_dataset

ds = load_dataset("nyu-mll/glue", "cola", split="train")

cola = ds.to_pandas()

acceptable_sentences = cola[cola['label'] == 1]['sentence'][0:500]
unacceptable_sentences = cola[cola['label'] == 0]['sentence'][0:500]

print("Processing acceptable sentences...")
acceptable_tda = process_texts(acceptable_sentences)
print("Processing unacceptable sentences...")
unacceptable_tda = process_texts(unacceptable_sentences)

Processing acceptable sentences...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

  1%|          | 5/500 [00:00<00:22, 21.58it/s]

Attention shape: (17, 17), min: 0.000000, max: 0.474609, mean: 0.052704
Using percentile-based threshold: 0.009781 (50th percentile)
Non-zero values: 136/136 (100.0%)
Value range: [0.002300, 0.474609]
Graph created: 17 nodes, 68 edges
Attention shape: (12, 12), min: 0.000000, max: 0.475098, mean: 0.072571
Using percentile-based threshold: 0.012878 (50th percentile)
Non-zero values: 66/66 (100.0%)
Value range: [0.003695, 0.475098]
Graph created: 12 nodes, 33 edges
Attention shape: (12, 12), min: 0.000000, max: 0.475098, mean: 0.072571
Using percentile-based threshold: 0.013565 (50th percentile)
Non-zero values: 66/66 (100.0%)
Value range: [0.003517, 0.475098]
Graph created: 12 nodes, 33 edges
Attention shape: (13, 13), min: 0.000000, max: 0.468506, mean: 0.067261
Using percentile-based threshold: 0.011932 (50th percentile)
Non-zero values: 78/78 (100.0%)
Value range: [0.002790, 0.468506]
Graph created: 13 nodes, 39 edges
Attention shape: (12, 12), min: 0.000000, max: 0.480225, mean: 0.0

  2%|▏         | 8/500 [00:00<00:22, 21.90it/s]

...

Attention shape: (16, 16), min: 0.000000, max: 0.475098, mean: 0.055664
Using percentile-based threshold: 0.008987 (50th percentile)
Non-zero values: 120/120 (100.0%)
Value range: [0.001543, 0.475098]
Graph created: 16 nodes, 60 edges
Attention shape: (11, 11), min: 0.000000, max: 0.475830, mean: 0.078552
Using percentile-based threshold: 0.014259 (50th percentile)
Non-zero values: 55/55 (100.0%)
Value range: [0.002710, 0.475830]
Graph created: 11 nodes, 27 edges
Attention shape: (10, 10), min: 0.000000, max: 0.476562, mean: 0.085876
Using percentile-based threshold: 0.013824 (50th percentile)
Non-zero values: 45/45 (100.0%)
Value range: [0.005749, 0.476562]
Graph created: 10 nodes, 22 edges
Attention shape: (10, 10), min: 0.000000, max: 0.478027, mean: 0.085327
Using percentile-based threshold: 0.015686 (50th percentile)
Non-zero values: 45/45 (100.0%)
Value range: [0.003595, 0.478027]
Graph created: 10 nodes, 22 edges
Attention shape: (11, 11), min: 0.000000, max: 0.480225, mean: 0.0

100%|█████████▉| 498/500 [00:22<00:00, 22.07it/s]

Attention shape: (14, 14), min: 0.000000, max: 0.468506, mean: 0.062988
Using percentile-based threshold: 0.010376 (50th percentile)
Non-zero values: 91/91 (100.0%)
Value range: [0.002392, 0.468506]
Graph created: 14 nodes, 45 edges
Attention shape: (11, 11), min: 0.000000, max: 0.479492, mean: 0.078735
Using percentile-based threshold: 0.012474 (50th percentile)
Non-zero values: 55/55 (100.0%)
Value range: [0.003614, 0.479492]
Graph created: 11 nodes, 27 edges
Attention shape: (10, 10), min: 0.000000, max: 0.477051, mean: 0.084961
Using percentile-based threshold: 0.016846 (50th percentile)
Non-zero values: 45/45 (100.0%)
Value range: [0.006634, 0.477051]
Graph created: 10 nodes, 22 edges
Attention shape: (19, 19), min: 0.000000, max: 0.479492, mean: 0.047485
Using percentile-based threshold: 0.007351 (50th percentile)
Non-zero values: 171/171 (100.0%)
Value range: [0.000886, 0.479492]
Graph created: 19 nodes, 85 edges
Attention shape: (14, 14), min: 0.000000, max: 0.468506, mean: 0.0

100%|██████████| 500/500 [00:22<00:00, 22.37it/s]

Attention shape: (16, 16), min: 0.000000, max: 0.468506, mean: 0.055389
Using percentile-based threshold: 0.009415 (50th percentile)
Non-zero values: 120/120 (100.0%)
Value range: [0.002113, 0.468506]
Graph created: 16 nodes, 60 edges
Attention shape: (21, 21), min: 0.000000, max: 0.468506, mean: 0.042908
Using percentile-based threshold: 0.007393 (50th percentile)
Non-zero values: 210/210 (100.0%)
Value range: [0.001662, 0.468506]
Graph created: 21 nodes, 105 edges


In [28]:
!pip install textdescriptives spacy
!python -m spacy download en_core_web_sm

import spacy
import textdescriptives as td
from tqdm import tqdm
import pandas as pd

# Load spacy model
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textdescriptives/all")

def get_desc(data):
    """Extract linguistic features from text data"""
    doc = nlp(data.iloc[0])
    df1 = td.extract_df(doc)
    
    for text in tqdm(data.iloc[1:]):
        text = nlp(text)
        df2 = td.extract_df(text)
        df1 = df1._append(df2)
    
    return df1

# extract features for CoLA
print("Extracting linguistic features for acceptable sentences...")
acceptable_feat = get_desc(acceptable_sentences)
print("Extracting linguistic features for unacceptable sentences...")
unacceptable_feat = get_desc(unacceptable_sentences)

# Define feature categories
feature_categories = {
    'token_stats': [
        'token_length_mean', 'token_length_median', 'token_length_std',
        'n_tokens', 'n_unique_tokens', 'proportion_unique_tokens',
        'mean_word_length', 'alpha_ratio'
    ],
    
    'sentence_stats': [
        'sentence_length_mean', 'sentence_length_median', 'sentence_length_std',
        'n_sentences', 'doc_length'
    ],
    
    'syllable_stats': [
        'syllables_per_token_mean', 'syllables_per_token_median', 
        'syllables_per_token_std'
    ],
    
    'pos_proportions': [
        'pos_prop_ADJ', 'pos_prop_ADP', 'pos_prop_ADV', 'pos_prop_AUX',
        'pos_prop_CCONJ', 'pos_prop_DET', 'pos_prop_INTJ', 'pos_prop_NOUN',
        'pos_prop_NUM', 'pos_prop_PART', 'pos_prop_PRON', 'pos_prop_PROPN',
        'pos_prop_PUNCT', 'pos_prop_SCONJ', 'pos_prop_SYM', 'pos_prop_VERB',
        'pos_prop_X'
    ],
    
    'information_theory': [
        'entropy', 'perplexity', 'per_word_perplexity', 'oov_ratio'
    ],
    
    'coherence': [
        'first_order_coherence', 'second_order_coherence'
    ],
    
    'dependency': [
        'dependency_distance_mean', 'dependency_distance_std',
        'prop_adjacent_dependency_relation_mean', 
        'prop_adjacent_dependency_relation_std'
    ],
    
    'readability': [
        'flesch_reading_ease', 'flesch_kincaid_grade', 'smog',
        'gunning_fog', 'automated_readability_index', 
        'coleman_liau_index', 'lix', 'rix'
    ],
    
    'quality_duplicates': [
        'passed_quality_check', 'duplicate_line_chr_fraction',
        'duplicate_paragraph_chr_fraction', 'duplicate_ngram_chr_fraction_5',
        'duplicate_ngram_chr_fraction_6', 'duplicate_ngram_chr_fraction_7',
        'duplicate_ngram_chr_fraction_8', 'duplicate_ngram_chr_fraction_9',
        'duplicate_ngram_chr_fraction_10'
    ],
    
    'ngram_stats': [
        'top_ngram_chr_fraction_2', 'top_ngram_chr_fraction_3',
        'top_ngram_chr_fraction_4'
    ],
    
    'formatting': [
        'n_characters', 'n_stop_words', 'symbol_to_word_ratio_#',
        'proportion_ellipsis', 'proportion_bullet_points', 
        'contains_lorem ipsum'
    ]
}

def get_category_features(df, category_name, feature_list):
    available_features = [f for f in feature_list if f in df.columns]
    return df[available_features]

acceptable_categorized = {}
unacceptable_categorized = {}

for category, features in feature_categories.items():
    acceptable_categorized[category] = get_category_features(
        acceptable_feat, category, features
    )
    unacceptable_categorized[category] = get_category_features(
        unacceptable_feat, category, features
    )
    print(f"{category}: {len(acceptable_categorized[category].columns)} features")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached textdescriptives-2.8.4-py3-none-any.whl.metadata (25 kB)
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached pyphen-0.17.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached ftfy-6.3.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached spacy_lookups_data-1.0.5-py2.py3-none-any.whl.metadata (4.8 kB)
Using cached textdescriptives-2.8.4-py3-none-any.whl (254 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
Using cached ftfy-6.3.1-py3-none-any.whl (44 kB)
Using cached pyphen-0.17.2-py3-none-any.whl (2.1 MB)
Using cached spacy_lookups_data-1.0.5-py2.py3-none-any.whl (98.5 MB)
  Attempting uninstall: numpy━━━━━━━━━━━━━━━━━━━ 0/5 [spacy_lookups_data]
    Found existing installation: numpy 2.1.2 0/5 [spacy_lookups_data]
    Uninstalling numpy-2.1.2:╺━━━━━━━━━━━━━━━━━━━━━━━ 2/5 [numpy]
      Successfully uninstalled numpy-2.1.2m━━━━━━━━━━━━━━━━━━━━━━━ 2/5 [numpy]
   

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 84.2 MB/s  0:00:00 eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Could not load lexeme probability table for language en. This will
result in NaN values for perplexity and entropy.
Extracting linguistic features for acceptable sentences...


 12%|█▏        | 59/499 [00:00<00:03, 143.23it/s]/usr/local/lib/python3.12/dist-packages/textdescriptives/components/coherence.py:44: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Span.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarities.append(sent.similarity(sents[i + order]))
100%|██████████| 499/499 [00:03<00:00, 136.26it/s]


Extracting linguistic features for unacceptable sentences...


100%|██████████| 499/499 [00:03<00:00, 135.35it/s]

token_stats: 8 features
sentence_stats: 5 features
syllable_stats: 3 features
pos_proportions: 17 features
information_theory: 4 features
coherence: 2 features
dependency: 4 features
readability: 8 features
quality_duplicates: 9 features
ngram_stats: 3 features
formatting: 6 features


In [31]:
!pip install dcor

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dcor
from matplotlib.backends.backend_pdf import PdfPages

# Define TDA features to use
tda_feature_names = ["Num_0dim", "Max_0dim", "Mean_0dim", "betti_curve_0", "persistence_entropy_0"]

def clean_data(df):
    """Clean dataframe by removing NaN/inf and converting to float"""
    df_clean = df.copy()
    df_clean = df_clean.replace([np.inf, -np.inf], np.nan)
    df_clean = df_clean.fillna(df_clean.mean())
    df_clean = df_clean.fillna(0)
    # Ensure all values are float
    df_clean = df_clean.astype(float)
    return df_clean

# Function to compute distance correlation matrix
def compute_dcor_matrix(tda_data, linguistic_data):
    """Compute distance correlation between TDA and linguistic features"""
    # Clean data first
    tda_clean = clean_data(tda_data)
    ling_clean = clean_data(linguistic_data)
    
    tda_features = tda_clean.to_numpy()
    linguistic_features = ling_clean.to_numpy()
    
    corr_matrix = np.zeros((tda_features.shape[1], linguistic_features.shape[1]))
    for i in range(tda_features.shape[1]):
        for j in range(linguistic_features.shape[1]):
            try:
                corr_matrix[i, j] = dcor.distance_correlation(tda_features[:, i], linguistic_features[:, j])
            except Exception as e:
                print(f"Warning: Could not compute correlation for indices ({i}, {j}): {e}")
                corr_matrix[i, j] = 0.0
    
    return corr_matrix

# Function to plot heatmap for a category
def plot_category_heatmap(corr_matrix, linguistic_feature_names, title, figsize=(12, 6)):
    """Plot heatmap for a specific category"""
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt='.3f',
                xticklabels=linguistic_feature_names,
                yticklabels=tda_feature_names,
                vmin=0, vmax=1, ax=ax)  # Distance correlation is between 0 and 1
    ax.set_title(title, fontsize=14, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    return fig

# Extract and clean TDA features
focus_tda_acceptable = clean_data(acceptable_tda[tda_feature_names])
focus_tda_unacceptable = clean_data(unacceptable_tda[tda_feature_names])

# Create PDF file
pdf_filename = "CoLA_TDA_Linguistic_Heatmaps.pdf"
with PdfPages(pdf_filename) as pdf:
    # Process each category
    for category, features in feature_categories.items():
        print(f"\n{'='*60}")
        print(f"Processing category: {category.upper()}")
        print(f"{'='*60}")
        
        # Get available features for this category
        available_features = [f for f in features if f in acceptable_feat.columns and f in unacceptable_feat.columns]
        
        if len(available_features) == 0:
            print(f"No features available for {category}, skipping...")
            continue
        
        print(f"Features in this category: {len(available_features)}")
        
        # Extract features for this category
        acceptable_cat_feat = acceptable_feat[available_features]
        unacceptable_cat_feat = unacceptable_feat[available_features]
        
        # Check for data quality
        print(f"Acceptable features shape: {acceptable_cat_feat.shape}")
        print(f"Unacceptable features shape: {unacceptable_cat_feat.shape}")
        
        # Compute correlations for ACCEPTABLE sentences
        print(f"Computing correlations for acceptable sentences - {category}...")
        corr_matrix_acceptable = compute_dcor_matrix(focus_tda_acceptable, acceptable_cat_feat)
        
        # Plot heatmap for ACCEPTABLE sentences
        fig = plot_category_heatmap(
            corr_matrix_acceptable,
            available_features,
            f"CoLA Acceptable - {category.replace('_', ' ').title()}"
        )
        pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)
        
        # Compute correlations for UNACCEPTABLE sentences
        print(f"Computing correlations for unacceptable sentences - {category}...")
        corr_matrix_unacceptable = compute_dcor_matrix(focus_tda_unacceptable, unacceptable_cat_feat)
        
        # Plot heatmap for UNACCEPTABLE sentences
        fig = plot_category_heatmap(
            corr_matrix_unacceptable,
            available_features,
            f"CoLA Unacceptable - {category.replace('_', ' ').title()}"
        )
        pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)
        
        # Compute and plot DIFFERENCE heatmap
        print(f"Computing difference - {category}...")
        corr_matrix_diff = corr_matrix_acceptable - corr_matrix_unacceptable
        
        fig, ax = plt.subplots(figsize=(12, 6))
        sns.heatmap(corr_matrix_diff, annot=True, cmap="RdBu_r", fmt='.3f', center=0,
                    xticklabels=available_features,
                    yticklabels=tda_feature_names,
                    vmin=-1, vmax=1, ax=ax)
        ax.set_title(f"CoLA Difference: Acceptable - Unacceptable - {category.replace('_', ' ').title()}", 
                     fontsize=14, fontweight='bold')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)
        
        print(f"Completed {category}!")

print("\n" + "="*60)
print("ALL CATEGORIES COMPLETE!")
print(f"Heatmaps saved to: {pdf_filename}")
print("="*60)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



Processing category: TOKEN_STATS
Features in this category: 8
Acceptable features shape: (500, 8)
Unacceptable features shape: (500, 8)
Computing correlations for acceptable sentences - token_stats...
Computing correlations for unacceptable sentences - token_stats...
Computing difference - token_stats...
Completed token_stats!

Processing category: SENTENCE_STATS
Features in this category: 5
Acceptable features shape: (500, 5)
Unacceptable features shape: (500, 5)
Computing correlations for acceptable sentences - sentence_stats...
Computing correlations for unacceptable sentences - sentence_stats...
Computing difference - sentence_stats...
Completed sentence_stats!

Processing category: SYLLABLE_STATS
Features in this category: 3
Acceptable features shape: (500, 3)
Unacceptable features shape: (500, 3)
Computing correlations for acceptable sentences - syllable_stats...
Computing correlations for unacceptable sentences - syllable_stats...
Computing difference - syllable_stats...
Comple

/tmp/ipykernel_765/797005161.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean = df_clean.fillna(df_clean.mean())
/tmp/ipykernel_765/797005161.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean = df_clean.fillna(df_clean.mean())


Computing difference - information_theory...
Completed information_theory!

Processing category: COHERENCE
Features in this category: 2
Acceptable features shape: (500, 2)
Unacceptable features shape: (500, 2)
Computing correlations for acceptable sentences - coherence...
Computing correlations for unacceptable sentences - coherence...
Computing difference - coherence...
Completed coherence!

Processing category: DEPENDENCY
Features in this category: 4
Acceptable features shape: (500, 4)
Unacceptable features shape: (500, 4)
Computing correlations for acceptable sentences - dependency...
Computing correlations for unacceptable sentences - dependency...
Computing difference - dependency...
Completed dependency!

Processing category: READABILITY
Features in this category: 8
Acceptable features shape: (500, 8)
Unacceptable features shape: (500, 8)
Computing correlations for acceptable sentences - readability...
Computing correlations for unacceptable sentences - readability...
Computing d